# Numbers with uncertainties

The result of a measurement should be given as a number with an attached uncertainties, besides the physical unit, and all operations performed involving the result of the measurement should propagate the uncertainty, taking care of correlation between quantities.

There is a Julia package for dealing with numbers with uncertainties: [`Measurements.jl`](https://github.com/JuliaPhysics/Measurements.jl).  Thanks to Julia's features, `DifferentialEquations.jl` easily works together with `Measurements.jl` out-of-the-box.

This notebook will cover some of the examples from the tutorial about lassical Physics.

## Radioactive Decay of Carbon-14

The rate of decay of carbon-14 is governed by a first order linear ordinary differential equation

$$ \frac{\mathrm{d}u(t)}{\mathrm{d}t} = -c u(t) $$

where $c = (5730 \pm 40)$ years is the half-life of carbon-14.

In [2]:
using DifferentialEquations, Measurements

#Setup
u₀ = 1 ± 0
tspan = (0.0, 1.0)

#Define the problem
radioactivedecay(t,u) = -(5.730 ± 0.040)*u

#Pass to solver
prob = ODEProblem(radioactivedecay,u₀,tspan)
sol = solve(prob,Tsit5());

LoadError: [91mMethodError: Cannot `convert` an object of type Measurements.Measurement{Float64} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

Oh, no, we got an error!

The problem lies in the fact that the extrema of the time interval in which we want to solve the equation, `tspan`, should be of type `Measurement` as well.

In [3]:
tspan = (0.0 ± 0, 1.0 ± 0)
prob = ODEProblem(radioactivedecay, u₀, tspan)
sol = solve(prob, Tsit5())

println("Quantity of carbon-14 after ",  Measurements.value(sol.t[10]) * 1000, " years: ", sol[10])

Quantity of carbon-14 after 780.1928441909264 years: 0.011442440679299993 ± 1.2277661385596172


Ok, now it works.  What... wait, the uncertainty looks too large!

Yes, here is a subtlety of `Measurements.jl` that you should be aware of:

In [4]:
5.23 ± 0.14 === 5.23 ± 0.14

false

In [5]:
(5.23± 0.14) - (5.23 ± 0.14)

0.0 ± 0.19798989873223333

In [6]:
(5.23 ± 0.14) / (5.23 ± 0.14)

1.0 ± 0.0378565771954557

The two numbers above, even though have the same nominal value and the same uncertainties, are actually two different measurements that only by chance share the same figures and their difference and their ratio have a non-zero uncertainty.  It is common in physics to get very similar, or even equal, results for a repeated measurement, but the two measurements are not the same thing.  This is also the reason why the uncertainty in the solution of the differential equation was so large without apparent reason.

However, if you define a variable:

In [7]:
x = 5.23 ± 0.14

5.23 ± 0.14

In [8]:
x === x

true

In [9]:
x - x

0.0 ± 0.0

In [10]:
x / x

1.0 ± 0.0

we have that `x` is strictly equal to itself and the difference between `x` and itself and its own ratio have zero uncertainty, because there is no doubt about the result of these operations.  It is this feature of `Measurements.jl` that enables it to correctly propagate the uncertainty of numbers accurately taking care of correlation between measurements.  However, you have also to remember it in order not to make mistakes.

### Complete correct version

The only thing we have to do to fix the error is to define a variable equal to the half-life of carbon-14:

In [14]:
using DifferentialEquations, Measurements

# Half-life of radiocarbon, in thousands of years
c = 5.730 ± 0.040

#Setup
u₀ = 1 ± 0
tspan = (0 ± 0, 1 ± 0)

#Define the problem
radioactivedecay(t,u) = -c * u

#Pass to solver
prob = ODEProblem(radioactivedecay, u₀, tspan)
sol = solve(prob, Tsit5())

println("Quantity of carbon-14 after ",  Measurements.value(sol.t[10]) * 1000, " years:")
println("Numerical: ", sol[10])
println("Analytic:  ", exp(-c * sol.t[10]))

Quantity of carbon-14 after 780.1928441909264 years:
Numerical: 0.011442440679299993 ± 5.653199063707093e-5
Analytic:  0.011441536447307409 ± 5.65314838858269e-5


Now the uncertainty is more reasonable and indeed it's very close to the analytic solution.

## Simple pendulum

### Small angles approximation

The next problem we are going to study is the simple pendulum in the approximation of small angles.  We address this simplified case because there exists an easy analytic solution to compare.

The differential equation we want to solve is

$$ \ddot{\theta} + \frac{g}{L} \theta = 0 $$

where $g = (9.79 \pm 0.02)~\mathrm{m}/\mathrm{s}^2$ is the gravitational acceleration measured where the experiment is carried out, and $L = (1.00 \pm 0.01)~\mathrm{m}$ is the length of the pendulum.

When you set up the problem for `DifferentialEquations.jl` remember to define the measurements as variables, as seen above.

In [12]:
using DifferentialEquations, Measurements

g = 9.79 ± 0.02; # Gravitational constants
L = 1.00 ± 0.01; # Length of the pendulum

#Initial Conditions
u₀ = [0 ± 0, π / 60 ± 0.01] # Initial speed and initial angle
tspan = (0 ± 0, 6.3 ± 0)

#Define the problem
function simplependulum(t, u, du)
    θ  = u[1]
    dθ = u[2]
    du[1] = dθ
    du[2] = -(g/L)*θ
end

#Pass to solvers
prob = ODEProblem(simplependulum, u₀, tspan)
sol = solve(prob, Tsit5())

# Analytic solution
u = u₀[2] .* cos.(sqrt(g / L) .* sol.t)

# Compare the results
for i in eachindex(sol)
    println("At time ", Measurements.value(sol.t[i]), ":")
    println("Numerical: ", sol[i][2])
    println("Analytic:  ", u[i], "\n")
end

At time 0.0:
Numerical: 0.05235987755982988 ± 0.01
Analytic:  0.05235987755982988 ± 0.01

At time 0.01874067268761529:
Numerical: 0.05226988694319587 ± 0.010016533129059067
Analytic:  0.05226988694328826 ± 0.010016533128972755

At time 0.07031705589571197:
Numerical: 0.051097701052042824 ± 0.01003703743455236
Analytic:  0.05109770109446497 ± 0.010037037421001361

At time 0.14906367826316927:
Numerical: 0.046767352820252184 ± 0.009751532041375943
Analytic:  0.046767353505391254 ± 0.009751531967149979

At time 0.247808618808041:
Numerical: 0.03739351687540871 ± 0.008956746341327032
Analytic:  0.03739352078031407 ± 0.008956745836610488

At time 0.37407742478501255:
Numerical: 0.020406578416927567 ± 0.006907076443448302
Analytic:  0.020406600320601386 ± 0.006907075215268615

At time 0.5203959060333951:
Numerical: -0.0030074930875202337 ± 0.0032168350863306947
Analytic:  -0.00300742203920318 ± 0.0032168365645930076

At time 0.673839216519664:
Numerical: -0.026811345454262146 ± 0.00187642505

Great, this time we got the right result at the first take!

## Arbitrary amplitude

Now that we know how to solve differential equations involving numbers with uncertainties we can solve the simple pendulum problem without any approximation.  This time the differential equation to solve is the following:

$$ \ddot{\theta} + \frac{g}{L} \sin(\theta) = 0 $$

In [13]:
using DifferentialEquations, Measurements

g = 9.79 ± 0.02; # Gravitational constants
L = 1.00 ± 0.01; # Length of the pendulum

#Initial Conditions
u₀ = [0 ± 0, π / 3 ± 0.02] # Initial speed and initial angle
tspan = (0 ± 0, 6.3 ± 0)

#Define the problem
function simplependulum(t, u, du)
    θ  = u[1]
    dθ = u[2]
    du[1] = dθ
    du[2] = -(g/L) * sin(θ)
end

#Pass to solvers
prob = ODEProblem(simplependulum, u₀, tspan)
sol = solve(prob, Tsit5())

# Compare the results
for i in eachindex(sol)
    println("At time ", Measurements.value(sol.t[i]), ": \n", sol[i][2], "\n")
end

At time 0.0: 
1.0471975511965976 ± 0.02

At time 0.0009540186378592696: 
1.0471928857344095 ± 0.020000088933675487

At time 0.010494205016451964: 
1.0466330862031832 ± 0.02001075881181086

At time 0.04602000795155588: 
1.036362258987595 ± 0.020029346729207014

At time 0.11023197054203128: 
0.9855918876612995 ± 0.019680485450011772

At time 0.1967352994364424: 
0.8555913479439925 ± 0.018193636007628352

At time 0.3085823344124181: 
0.5988371548122635 ± 0.014852260345256862

At time 0.4460254171260305: 
0.19139554882837082 ± 0.008436002307096398

At time 0.5989010921583777: 
-0.2973685920626386 ± 0.001556577033282183

At time 0.7532414184491362: 
-0.7258472903668225 ± 0.009143642348421929

At time 0.9484916071090639: 
-1.0271151850828892 ± 0.018261225560948648

At time 1.1495658868481962: 
-0.9507361301315439 ± 0.021113729952978316

At time 1.348302170989763: 
-0.5206848391873202 ± 0.017568199981165895

At time 1.583599246375559: 
0.21436322480331002 ± 0.005364684846828113

At time 1.803